In [1]:
import cv2
import os

# Extract frames from video
def extract_frames(video_path, frames_dir):
    vidcap = cv2.VideoCapture(video_path)
    success, image = vidcap.read()
    count = 0
    while success:
        cv2.imwrite(os.path.join(frames_dir, f"frame{count}.jpg"), image)  # save frame as JPEG file      
        success, image = vidcap.read()
        count += 1
    vidcap.release()
    return count  # Return the number of frames

# Usage for frame extraction
def extract_frames_main(video_path, frames_dir):
    os.makedirs(frames_dir, exist_ok=True)
    extract_frames(video_path, frames_dir)

# Paths for frame extraction
video_path = r'D:\deeplabcut\\control_sal\322.mp4'  # Your video file path
frames_dir = r'D:\deeplabcut\\control_sal\322\frames'  # Directory to save frames

# Extract frames
extract_frames_main(video_path, frames_dir)


In [ ]:
import pandas as pd
import cv2
import os
from collections import defaultdict

# Read the B-SOiD labels
def read_bsoid_labels(csv_path):
    df = pd.read_csv(csv_path)
    return df['B-SOiD labels'].tolist()  # Adjust the column name if different

# Group frames by B-SOiD label
def group_frames_by_label(frames_dir, labels):
    frames_groups = defaultdict(list)
    for i, label in enumerate(labels):
        frame_path = os.path.join(frames_dir, f"frame{i}.jpg")
        if os.path.exists(frame_path):
            frames_groups[label].append(frame_path)
    return frames_groups

# Create videos from frames groups
def create_videos_from_groups(frames_groups, output_dir):
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'mp4v' for .mp4 files
    for label, frames in frames_groups.items():
        if frames:  # If there are frames for this label
            video_path = os.path.join(output_dir, f"322_sal_cluster{label}.mp4")
            # Assume that all frames have the same size
            height, width, layers = cv2.imread(frames[0]).shape
            video = cv2.VideoWriter(video_path, fourcc, 30, (width, height))
            for frame_path in frames:
                frame = cv2.imread(frame_path)
                video.write(frame)
            video.release()

# Usage for creating videos
def create_videos_main(frames_dir, csv_path, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    labels = read_bsoid_labels(csv_path)
    num_frames = len([name for name in os.listdir(frames_dir) if os.path.isfile(os.path.join(frames_dir, name))])
    print(f"Total frames found: {num_frames}")
    print(f"Total labels found: {len(labels) - 2}")

    # Ensuring that we have a label for each frame
    if num_frames != len(labels) - 2:
        print("Warning: The number of frames does not match the number of labels!")
    else:
        print("The number of frames match the number of labels!")

    frames_groups = group_frames_by_label(frames_dir, labels[:num_frames])
    create_videos_from_groups(frames_groups, output_dir)

# Paths for video creation
video_path = r'D:\deeplabcut\\control_sal\322.mp4'  # Your video file path
frames_dir = r'D:\deeplabcut\\control_sal\322\frames'  # Directory to save frames
csv_path = r'D:\deeplabcut\control_sal\BSOID\Oct-27-2023labels_pose_30Hz322DLC_resnet50_SP_behaviorOct10shuffle1_800000.csv'  # Your CSV file path
output_dir = r'D:\deeplabcut\control_sal\322\output_videos'  # Directory to save output videos

# Create videos
create_videos_main(frames_dir, csv_path, output_dir)


C:\Users\huang\AppData\Local\Temp\ipykernel_1104\2513034842.py:8: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


Total frames found: 65294
Total labels found: 65294
The number of frames match the number of labels!
